In [4]:
from collections import defaultdict
import json

train_path = '/opt/ml/detection/dataset/'
test_path = '/opt/ml/detection/dataset/test.json'
best_predictions_path = 'train.json' # val.py를 돌려서 나온 JSON 경로

score_thres = 0.6 # score threshold로 일정 점수 이상의 box값만 정답으로써 반영한다.

with open (train_path,'r') as f:
    train = json.load(f)
    
with open(test_path,'r') as f:
    test = json.load(f)
    
with open(best_predictions_path,'r') as f:
    best = json.load(f)
    preds = defaultdict(list)
    for i in best:
        preds[i['image_id']].append(i)
        
img_id = 4883 # Test 이미지가 저장될 시작 번호
anno_id = 23144 # Test annotations가 저장될 시작 번호

for test_img in test['images']:
    
    new_annotations = []
    
    for annotations in preds[test_img['id']]:
        if annotations['score'] > score_thres: 

            x_min,y_min,x_max,y_max = annotations['bbox']
            x_min = round(x_min,1)
            y_min = round(y_min,1)
            x_max = round(x_max,1)
            y_max = round(y_max,1)
            new = {
                'image_id' : img_id,
                'category_id': annotations['category_id'],
                'area': round(x_max*y_max,2),
                'bbox':[x_min,y_min,x_max,y_max],
                'iscrowd': 0,
                'id': anno_id
            }
            new_annotations.append(new)
            anno_id+=1
    
    if len(new_annotations): # 예측결과값이 있는 경우에만 추가해준다.
        test['id'] = img_id
        train['images'].append(test_img)
        train['annotations'].extend(new_annotations)
        img_id+=1

with open ('pseudo.json','w') as f:
    json.dump(train,f)
    


